In [12]:
import pandas as pd
import seaborn as sb
import numpy as np
import math as m
from matplotlib import pyplot as plt

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

# CSV file parsing

dataset_name = "seed_1"
seed_folder_path = f"data/seed/{dataset_name}/"
info_filename = f"{seed_folder_path}info.csv"
ratings_filename = f"{seed_folder_path}ratings.csv"
authors_filename = f"{seed_folder_path}authors.csv"

info = pd.read_csv(info_filename)
paper_authors = pd.read_csv(authors_filename)
paper_authors = paper_authors.values
paper_ratings = pd.read_csv(ratings_filename)
paper_ratings = paper_ratings.values

# Initial setup

dataset_name = info["Dataset"][0] 
papers_number = info["Paper"][0]
readers_number = info["Reader"][0] 
ratings_number = info["Rating"][0]
papers = np.arange(papers_number)
readers = np.arange(readers_number)
ratings = np.arange(ratings_number)
paper_score = np.zeros(papers_number)
rating_informativeness = np.zeros(ratings_number)
rating_accuracy_loss = np.zeros(ratings_number)
rating_bonus = np.zeros(ratings_number)
reader_bonus = np.zeros(readers_number)
reader_score = np.zeros(readers_number)


def quadratic_loss(a, b):
    return m.pow((a - b), 2)


def logistic_function(value):
    return 1 / 1 + (m.exp((-1 * (value - 0.5))))


for current_paper in papers:

    current_paper_ratings = []
    ratings_sum = 0
    
    # For each paper, consider only its ratings and throw away the other ones
    
    for index, entry in enumerate(paper_ratings):
    
        # Example: <1,1,2,0.8>
        # At Timestamp 1 Reader 1 gave to Paper 2 a Rating of 0.8
        timestamp = int(entry[0])
        reader = int(entry[1])
        paper = int(entry[2])
        rating = entry[3]
        
        if paper == current_paper:
            current_paper_ratings.append(entry)

    # For each rating of the paper under consideration, compute the required quantities
     
    for index, entry in enumerate(current_paper_ratings):

        # Example: <1,1,2,0.8>
        # At Timestamp 1 Reader 1 gave to Paper 2 a Rating of 0.8
        timestamp = int(entry[0])
        reader = int(entry[1])
        paper = int(entry[2])
        rating = entry[3]
    
        # 0 < i < n
    
        if 0 < index < len(current_paper_ratings)-1:
            
            # COMPUTATION START: QI_PAST
            
            past_ratings = current_paper_ratings[:index] 
            past_scores = []
            for past_index, past_entry in enumerate(past_ratings):
                past_rating = past_entry[3]
                past_scores.append(past_rating)
                
            # COMPUTATION START: QI_FUTURE
                
            future_ratings = current_paper_ratings[(index+1):]
            future_scores = []
            for future_index, future_entry in enumerate(future_ratings):
                future_rating = future_entry[3]
                future_scores.append(future_rating)
            
            qi_past_ratings = sum(past_scores) / len(past_scores)
            qi_future_ratings = sum(future_scores) / len(future_scores)
                                
            # COMPUTATION START: INFORMATIVENESS and ACCURACY LOSS
            
            rating_informativeness[timestamp] = quadratic_loss(qi_past_ratings, qi_future_ratings)
            rating_accuracy_loss[timestamp] = quadratic_loss(rating, qi_future_ratings)
            
            # COMPUTATION START: RATING BONUS
            
            rating_bonus[timestamp] = rating_informativeness[timestamp] * logistic_function(rating_accuracy_loss[timestamp])
            
            # COMPUTATION START: READER BONUS - it is the sum of the bonus computed for each of its ratings
            
            reader_bonus[reader] = reader_bonus[reader] + rating_bonus[timestamp]
            
        # Sum the current rating to compute the mean at the end
            
        ratings_sum = ratings_sum + rating
            
    # COMPUTATION START: PAPER SCORE - scores can be aggregated with an index of your choice
    
    paper_score[current_paper] = ratings_sum / len(current_paper_ratings)

In [13]:
# Summary

print("RATING INFORMATIVENESS: ", rating_informativeness)
print("RATING ACCURACY LOSS:   ", rating_accuracy_loss)
print("RATING BONUS:           ", rating_bonus)
print("READER BONUS:           ", reader_bonus)
print("PAPER  SCORE:           ", paper_score)


RATING INFORMATIVENESS:  [0.00 0.00 0.00 0.20 0.21 0.00 0.00 0.11 0.01 0.03 0.07 0.12 0.00 0.16
 0.11 0.07 0.00 0.20 0.01 0.16 0.01 0.01 0.00 0.00 0.01 0.12 0.01 0.01
 0.02 0.00 0.02 0.06 0.11 0.02 0.22 0.00 0.04 0.26 0.26 0.19 0.02 0.00
 0.00 0.00 0.00 0.00 0.00 0.00 0.09 0.00]
RATING ACCURACY LOSS:    [0.00 0.00 0.00 0.23 0.00 0.00 0.00 0.00 0.20 0.13 0.03 0.07 0.05 0.18
 0.04 0.03 0.39 0.07 0.08 0.02 0.26 0.00 0.03 0.13 0.18 0.01 0.09 0.38
 0.04 0.01 0.11 0.03 0.03 0.22 0.38 0.00 0.28 0.00 0.55 0.00 0.04 0.00
 0.02 0.00 0.19 0.06 0.00 0.00 0.38 0.00]
RATING BONUS:            [0.00 0.00 0.00 0.47 0.56 0.00 0.00 0.29 0.02 0.08 0.18 0.31 0.01 0.38
 0.28 0.18 0.01 0.50 0.03 0.43 0.02 0.02 0.01 0.00 0.02 0.33 0.01 0.02
 0.04 0.01 0.04 0.15 0.29 0.04 0.46 0.00 0.09 0.68 0.50 0.49 0.05 0.00
 0.01 0.00 0.01 0.00 0.00 0.00 0.19 0.00]
READER BONUS:            [0.83 0.84 0.67 0.89 0.98 0.83 0.41 0.52 0.60 0.63]
PAPER  SCORE:            [0.33 0.52 0.53 0.45 0.51]
